In [22]:
import pandas as pd
import numpy as np
df = pd.read_csv("Avalanche_Activity.csv")
df.head()

,danger_level,elevation,temperature,wind_speed,humidity,avalanche_act,snow_depth
0,60.0,1980,8,34,58,1,3.113
1,40.0,2412,11,11,58,0,12.560
2,40.0,2520,-7,40,60,0,13.320
3,40.0,2338,11,29,62,0,12.090
4,60.0,2270,-32,27,63,0,14.840


In [23]:
df.shape

(30867, 7)

In [24]:
import collections
print("Class Distribution:", collections.Counter(df['avalanche_act']))

Class Distribution: Counter({0: 24585, 1: 6282})


In [25]:
from imblearn.over_sampling import SMOTE
from collections import Counter

features = ["elevation", "temperature", "wind_speed", "humidity", "snow_depth"]
target = "avalanche_act"

X = df[features]
y = df[target]

smote = SMOTE(sampling_strategy=0.5, random_state=42)  
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Resampled Class Distribution:", Counter(y_resampled))


Resampled Class Distribution: Counter({0: 24585, 1: 12292})


C:\Users\tariq.aziz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

joblib.dump(scaler, "avalanch_scaler.pkl")

avalanche_model = RandomForestClassifier(n_estimators=100, random_state=42)
avalanche_model.fit(X_train_scaled, y_train)

joblib.dump(avalanche_model, "avalanch_rf_model.pkl")

['avalanch_rf_model.pkl']

In [27]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = avalanche_model.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8469360086767896
Confusion Matrix:
 [[4539  378]
 [ 751 1708]]
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.92      0.89      4917
           1       0.82      0.69      0.75      2459

    accuracy                           0.85      7376
   macro avg       0.84      0.81      0.82      7376
weighted avg       0.84      0.85      0.84      7376



In [31]:
# avalanche_model = joblib.load(r"C:\Users\tariq.aziz\OneDrive - University of Central Asia\Documents\FYYYYYPPPP\avalanch_rf_model.pkl")
# avalanche_scaler = joblib.load(r"C:\Users\tariq.aziz\OneDrive - University of Central Asia\Documents\FYYYYYPPPP\avalanch_scaler.pkl")

new_data = np.array([[2300, -3, 24, 4, 29]])

new_data_scaled = avalanche_scaler.transform(new_data)

prediction = avalanche_model.predict(new_data_scaled)
print("Predicted Avalanche Risk Level:", prediction[0])


Predicted Avalanche Risk Level: 0


C:\Users\tariq.aziz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
